### Differentiation on Linear Subspaces of G

In [1]:
from kingdon import Algebra
from gc_utils import *
import numpy as np
alg = Algebra(1,3)
locals().update(alg.blades)

In [2]:
#2.1
A = create_random_multivector(alg)
B = create_random_multivector(alg)
F = lambda X: (X+B)**3
X = create_random_multivector(alg)
h = 6e-6 # leave h as a global var, so to modify with ease

# central scheme, modify h to minimize truncation/round-off error
# a measure along a line
def differential(F, X, A):
    d = (h)*A
    return (1/(2*h))*F(X+d) - (1/(2*h))*F(X-d)

In [3]:
# a suggestion of h
def auto_step_size(x):
    """
    Automatically determine step size based on variable magnitude and machine epsilon.
    """
    eps = np.finfo(float).eps
    return np.maximum(np.abs(x) * np.cbrt(eps), 100 * eps)
X = create_random_multivector(alg)
np.mean(auto_step_size(X[:]))

3.8077052309934643e-06

In [4]:
#2.3
A2 = create_r_blade(2, alg)
F = lambda X: P(X, A2)**2
max_diff(differential(F, X, A), differential(F, P(X,A2), A))

4.0745362639427185e-10

In [5]:
#2.7 linearity
G = lambda X: (X+A2)**3
differential(lambda X: A*F(X) + G(X), X, B), A * differential(F, X, B) + differential(G,X,B)

(-33.0 + -30.6 𝐞₁ + -19.7 𝐞₂ + -5.05 𝐞₃ + -13.6 𝐞₄ + -18.4 𝐞₁₂ + 15.6 𝐞₁₃ + -14.9 𝐞₁₄ + 4.71 𝐞₂₃ + 4.08 𝐞₂₄ + -10.1 𝐞₃₄ + -13.1 𝐞₁₂₃ + 3.55 𝐞₁₂₄ + -2.39 𝐞₁₃₄ + -5.63 𝐞₂₃₄ + -2.09 𝐞₁₂₃₄,
 -33.0 + -30.6 𝐞₁ + -19.7 𝐞₂ + -5.05 𝐞₃ + -13.6 𝐞₄ + -18.4 𝐞₁₂ + 15.6 𝐞₁₃ + -14.9 𝐞₁₄ + 4.71 𝐞₂₃ + 4.08 𝐞₂₄ + -10.1 𝐞₃₄ + -13.1 𝐞₁₂₃ + 3.55 𝐞₁₂₄ + -2.39 𝐞₁₃₄ + -5.63 𝐞₂₃₄ + -2.09 𝐞₁₂₃₄)

In [6]:
#2.8 function product
differential(lambda X: F(X)*G(X), X, A), differential(F, X, A) * G(X) + F(X) * differential(G, X, A)

(2.93e+02 + 92.2 𝐞₁ + 35.9 𝐞₂ + -76.1 𝐞₃ + 65.4 𝐞₄ + 1.84e+02 𝐞₁₂ + -2.5e+02 𝐞₁₃ + 58.4 𝐞₁₄ + -65.8 𝐞₂₃ + -37.0 𝐞₂₄ + 1.61e+02 𝐞₃₄ + -77.3 𝐞₁₂₃ + 57.0 𝐞₁₂₄ + 5.74 𝐞₁₃₄ + -83.8 𝐞₂₃₄ + 29.4 𝐞₁₂₃₄,
 2.93e+02 + 92.2 𝐞₁ + 35.9 𝐞₂ + -76.1 𝐞₃ + 65.4 𝐞₄ + 1.84e+02 𝐞₁₂ + -2.5e+02 𝐞₁₃ + 58.4 𝐞₁₄ + -65.8 𝐞₂₃ + -37.0 𝐞₂₄ + 1.61e+02 𝐞₃₄ + -77.3 𝐞₁₂₃ + 57.0 𝐞₁₂₄ + 5.74 𝐞₁₃₄ + -83.8 𝐞₂₃₄ + 29.4 𝐞₁₂₃₄)

In [7]:
#2.9
h = 6e-6
GF = lambda X: G(F(X))
max_diff(differential(GF, X, A), differential(G, F(X), differential(F, X, A)))

3.073364496231079e-08

In [8]:
#2.12
h = 6e-6
B = create_random_multivector(alg)
FA = lambda X: differential(F, X, A)
FB = lambda X: differential(F, X, B)
FAB = differential(FA, X, B)
FBA = differential(FB, X, A)
max_diff(FAB, FBA)

3.637978807091713e-11

In [9]:
# a measure around a point X, reciprocal frame assures each direction measured evenly
def derivative(F, X, alg, grade=None, frame=None, r_frame=None):
    if not frame:
        if grade or (grade == 0):
            frame = r_vector_frame(alg.frame, grade)
            r_frame = r_vector_frame(reciprocal(alg.frame), grade, reverse=True)
        else:
            frame = multi_frame(alg.frame)
            r_frame = reci_frame(alg.frame)
    dF = 0
    for v, r in zip(frame, r_frame):
        dF += r * (differential(F, X, v))
    return dF


In [10]:
# Can we recover differential from derivative? Yes, but in the sense that the derivative is invertible. That's integration!
# Always remember these two are not equal!
A.sp(derivative(F, X, alg)), differential(F, X, A)

(-9.82,
 -5.66 + 1.52 𝐞₁ + 2.15 𝐞₂ + 0.559 𝐞₃ + 1.78 𝐞₄ + -4.31 𝐞₁₂ + 1.52 𝐞₁₃ + -0.415 𝐞₁₄ + 3.74 𝐞₂₃ + 4.46 𝐞₂₄ + -1.94 𝐞₃₄ + 2.14e-11 𝐞₁₂₃ + 2.43e-11 𝐞₁₂₄ + -1.07e-11 𝐞₁₃₄ + 9.25e-12 𝐞₂₃₄ + 1.85e-11 𝐞₁₂₃₄)

In [16]:
# Yet we have a equality (similar to 2.19) for the scalar part
# The classical gradient
G = lambda X: F(X).grade(0)
A.sp(derivative(G, X, alg)), differential(G, X, A)

(-5.66, -5.66)

In [15]:
#2.13
# Randomly generate several frames to see dF is independent of frame
h = 6e-6
r = 4
dFs = []
for _ in range(5):
    basis = create_r_vectors(r, alg)
    r_basis = reciprocal(basis)
    frame = multi_frame(basis)
    r_frame = multi_frame(r_basis, reverse=True)

    dF = derivative(F, X, alg, frame=frame, r_frame=r_frame)
    dFs.append(dF)
[max_diff(dFs[-1], dF) for dF in dFs[:-1]]

[9.119374144717085e-10,
 1.0360779456597413e-09,
 6.152931497638292e-10,
 7.470519758356886e-10]

In [16]:
# For F defined on A2, we can restrict our bases
# choose a nonorthonormal-to-A2-vector e.g. e1
v1 = P(e1, A2)
# contract the parallel part
v2 = v1 | A2
frame = multi_frame([v1, v2])
r_frame = reci_frame([v1, v2])
dF = derivative(F, X, alg, frame=frame, r_frame=r_frame)
dFs[-1], dF

(3.73 + 1.85 𝐞₁ + 1.9 𝐞₂ + 2.23 𝐞₃ + 1.43 𝐞₄ + -0.104 𝐞₁₂ + 0.618 𝐞₁₃ + -0.358 𝐞₁₄ + 0.759 𝐞₂₃ + -0.287 𝐞₂₄ + -0.908 𝐞₃₄ + -1.65e-10 𝐞₁₂₃ + -3.98e-10 𝐞₁₂₄ + -1.34e-10 𝐞₁₃₄ + 4.07e-10 𝐞₂₃₄ + -3.51e-10 𝐞₁₂₃₄,
 3.73 + 1.85 𝐞₁ + 1.9 𝐞₂ + 2.23 𝐞₃ + 1.43 𝐞₄ + -0.104 𝐞₁₂ + 0.618 𝐞₁₃ + -0.358 𝐞₁₄ + 0.759 𝐞₂₃ + -0.287 𝐞₂₄ + -0.908 𝐞₃₄ + -4.38e-11 𝐞₁₂₃ + 2.9e-11 𝐞₁₂₄ + 1.72e-11 𝐞₁₃₄ + -4.66e-11 𝐞₂₃₄ + -2.78e-11 𝐞₁₂₃₄)

In [17]:
#2.18
# double differentials lower accuracy
# Should always replace d_F_ with dF in computation
# but if we just want to see the equivalence, we can just choose a fairly large h
h = 1e-2
F_ = lambda A: differential(F, X, A)
d_F_ = derivative(F_, A, alg, frame=frame, r_frame=r_frame)
# dF = derivative(F , X, alg, frame=frame, r_frame=r_frame)
max_diff(d_F_, dF), d_F_, dF

(4.817124477085599e-11,
 3.73 + 1.85 𝐞₁ + 1.9 𝐞₂ + 2.23 𝐞₃ + 1.43 𝐞₄ + -0.104 𝐞₁₂ + 0.618 𝐞₁₃ + -0.358 𝐞₁₄ + 0.759 𝐞₂₃ + -0.287 𝐞₂₄ + -0.908 𝐞₃₄ + -2.2e-12 𝐞₁₂₃ + 1.45e-12 𝐞₁₂₄ + 1.84e-12 𝐞₁₃₄ + -1.06e-12 𝐞₂₃₄ + 1.22e-12 𝐞₁₂₃₄,
 3.73 + 1.85 𝐞₁ + 1.9 𝐞₂ + 2.23 𝐞₃ + 1.43 𝐞₄ + -0.104 𝐞₁₂ + 0.618 𝐞₁₃ + -0.358 𝐞₁₄ + 0.759 𝐞₂₃ + -0.287 𝐞₂₄ + -0.908 𝐞₃₄ + -4.38e-11 𝐞₁₂₃ + 2.9e-11 𝐞₁₂₄ + 1.72e-11 𝐞₁₃₄ + -4.66e-11 𝐞₂₃₄ + -2.78e-11 𝐞₁₂₃₄)

In [20]:
# 2.19
h = 1e-5
def adjoint1(F, X, A, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = multi_frame(alg.frame)
        r_frame = reci_frame(alg.frame)
    _F = 0    
    for v, r in zip(frame, r_frame):
        _F += r * alg.sp(differential(F, X, v), A)
    return _F

# either works, but this one's slightly better
def adjoint(F, X, A, alg: Algebra, frame=None, r_frame=None):
    _F = derivative(lambda X: alg.sp(F(X), (A)), X, alg, frame=frame, r_frame=r_frame)
    return _F

_F = adjoint(F, X, A, alg)
_F1 = adjoint1(F, X, A, alg)
# The algebraic part always determine the order of operations.
# The scalar parts' order are arbitrary.
F_BA = differential(F, X, B).sp(A)
F_BA1 = differential(lambda X: F(X).sp(A), X, B)

B.sp(_F), B.sp(_F1), F_BA, F_BA1

(-2.71, -2.71, -2.71, -2.71)

In [20]:
frame = multi_frame(alg.frame)
r_frame = reci_frame(alg.frame)
F(X), sum(F(X).sp(A) * Ar for A, Ar in zip(frame, r_frame))

(-0.292 + 0.432 𝐞₁ + 0.444 𝐞₂ + 0.519 𝐞₃ + 0.334 𝐞₄ + -0.0242 𝐞₁₂ + 0.144 𝐞₁₃ + -0.0834 𝐞₁₄ + 0.177 𝐞₂₃ + -0.0668 𝐞₂₄ + -0.212 𝐞₃₄ + 1.39e-17 𝐞₁₂₃ + -2.08e-17 𝐞₁₂₄ + -9.71e-17 𝐞₁₃₄ + -8.33e-17 𝐞₂₃₄ + 6.07e-18 𝐞₁₂₃₄,
 -0.292 + 0.432 𝐞₁ + 0.444 𝐞₂ + 0.519 𝐞₃ + 0.334 𝐞₄ + -0.0242 𝐞₁₂ + 0.144 𝐞₁₃ + -0.0834 𝐞₁₄ + 0.177 𝐞₂₃ + -0.0668 𝐞₂₄ + -0.212 𝐞₃₄ + 1.39e-17 𝐞₁₂₃ + -2.08e-17 𝐞₁₂₄ + -9.71e-17 𝐞₁₃₄ + -8.33e-17 𝐞₂₃₄ + 6.07e-18 𝐞₁₂₃₄)

In [22]:
derivative(F, X, alg), sum(adjoint(F, X, A, alg) * Ar for A, Ar in zip(frame, r_frame))

(3.73 + 1.85 𝐞₁ + 1.9 𝐞₂ + 2.23 𝐞₃ + 1.43 𝐞₄ + -0.104 𝐞₁₂ + 0.618 𝐞₁₃ + -0.358 𝐞₁₄ + 0.759 𝐞₂₃ + -0.287 𝐞₂₄ + -0.908 𝐞₃₄ + -2.43e-12 𝐞₁₂₃ + 9.96e-12 𝐞₁₂₄ + 2.36e-12 𝐞₁₃₄ + -1.89e-11 𝐞₂₃₄ + -1.59e-12 𝐞₁₂₃₄,
 3.73 + 1.85 𝐞₁ + 1.9 𝐞₂ + 2.23 𝐞₃ + 1.43 𝐞₄ + -0.104 𝐞₁₂ + 0.618 𝐞₁₃ + -0.358 𝐞₁₄ + 0.759 𝐞₂₃ + -0.287 𝐞₂₄ + -0.908 𝐞₃₄ + -2.43e-12 𝐞₁₂₃ + 9.96e-12 𝐞₁₂₄ + 2.36e-12 𝐞₁₃₄ + -1.89e-11 𝐞₂₃₄ + -1.59e-12 𝐞₁₂₃₄)

In [45]:
#2.20 F defined on A2
max_diff(P(_F, A2), _F)

7.947836833110955e-11

In [46]:
#2.21
[max_diff(_F.grade(r), derivative(lambda X: F(X).sp(A), X, alg, r)) for r in range(0, 5)]

[0.0, 0.0, 0.0, 0.0, 0.0]

In [47]:
#2.24
r = 2
_sum = derivative(lambda X: F(X), X, alg) + derivative(lambda X: G(X), X, alg)
max_diff(derivative(lambda X: F(X) + G(X), X, alg), _sum)

1.2305889640629175e-10

In [48]:
#2.25a
_prod = derivative(lambda X: F(X), X, alg)*G(X) + derivative(lambda Y:F(X)*G(Y), X, alg)
max_diff(derivative(lambda X: F(X)*G(X), X, alg), _prod)

3.1390925414598314e-09

In [49]:
#2.25b The note by Hitzer regard this as a definition. But we shall see it's not! The operators have their meaning.
# r-derivative
grade = 2
frame = r_vector_frame(alg.frame, grade)
r_frame = r_vector_frame(reciprocal(alg.frame), grade, reverse=True)
FdG = 0
Fd = 0
dG = 0
for v, r in zip(frame, r_frame):
    FdG += differential(lambda X: F(X)*r*G(X), X, v) # the over dots made clear using lambda function
    Fd += differential(F, X, v) * r
    dG += r * differential(G, X, v)
max_diff(FdG, (Fd * G(X)) + (F(X)*dG))

1.275296312996943e-09

In [50]:
#2.25b derivative on entire frame
frame = multi_frame(alg.frame)
r_frame = reci_frame(alg.frame)
FdG = 0
Fd = 0
dG = 0
for v, r in zip(frame, r_frame):
    FdG += differential(lambda X: F(X)*r*G(X), X, v)
    Fd += differential(F, X, v) * r
    dG += r * differential(G, X, v)
max_diff(FdG, (Fd * G(X)) + (F(X)*dG))

2.0250467969162855e-09

In [51]:
#2.26 r-derivative
h = 1e-4
r = 2
max_diff(
derivative(GF, X, alg, r), 
derivative(lambda Y: differential(G, F(X), F(Y)), X, alg, r)
)

2.7164158566961305e-07

In [52]:
#2.26 derivative
h = 1e-4
r = None
max_diff(
derivative(GF, X, alg, r), 
derivative(lambda Y: differential(G, F(X), F(Y)), X, alg, r)
)

1.6725094269531837e-06

In [53]:
#2.28a
(
    max_diff(P(A, A2), differential(lambda X: P(X, A2), X, A)),
    max_diff(P(A, A2), derivative(lambda X: P(X, A2).sp(A), X, alg))
)

(1.1411982470121984e-12, 3.429673212096418e-12)

In [54]:
#2.28b
max_diff(P(A, A2).reverse(), differential(lambda X: P(X.reverse(), A2), X, A))

1.3818945987509323e-12

In [55]:
#2.29 d of n dim spade is 2^n
h = 1e-5
A3 = create_r_blade(3, alg)
(
derivative(lambda X: X, X, alg), #4d
derivative(lambda X: P(X, A3), X, alg), #3d
derivative(lambda X: P(X, A2), X, alg) #2d
)

(16.0,
 8.0 + 4.77e-12 𝐞₁₂ + 8.19e-12 𝐞₁₃ + -1.04e-11 𝐞₁₄ + -6.88e-12 𝐞₂₃ + -2.73e-12 𝐞₂₄ + 1.42e-11 𝐞₃₄,
 4.0 + -1.42e-11 𝐞₁₂ + 1.27e-11 𝐞₁₃ + 6.08e-12 𝐞₁₄ + -1e-11 𝐞₂₃ + 6.99e-12 𝐞₂₄ + 1.09e-11 𝐞₃₄ + -1.31e-11 𝐞₁₂₃₄)

In [56]:
#2.30
max_diff(derivative(lambda X: X.sp(X.reverse()), X, alg), 2*X.reverse())

1.5755174942455596e-11

In [57]:
#2.31
k=6
max_diff(differential(lambda X: X**k, X, A), sum((X**l)*A*(X**(k-1-l)) for l in range(k)))

1.2028749551973306e-07

In [58]:
#2.32 To allow different signature, we consider norm square instead
def normsq(X):
    return X.sp(X.reverse())[0]

k=3    
max_diff(derivative(lambda X: normsq(X) ** (k), X, alg), 2*k*normsq(X)**(k-1)*X.reverse())

1.242458358419185e-09

In [59]:
#2.33
max_diff(derivative(lambda X: np.log(np.abs(normsq(X))), X, alg), 2 * X.reverse()/normsq(X))

1.0502665404032996e-09

In [60]:
#2.30 var
for n in range(7):
    alg = Algebra(n)
    X = create_random_multivector(alg)
    dXXr = derivative(lambda X: X * X.reverse(), X, alg)
    Xr = X.reverse()
    ratio = np.round(dXXr[0]/Xr[0])
    print(ratio, max_diff(dXXr, ratio*Xr)) 

2.0 1.2004619520666893e-11
4.0 1.7052581569032554e-11
6.0 2.3314683517128287e-11
8.0 3.5776714923940744e-11
12.0 8.408918006352906e-11
24.0 2.1528556715111336e-10
56.0 3.9044323330017505e-10


Compare with 2.2.30. The derivative is also an integer multiple of $\tilde{X}$, yet a complex one:
$$ \partial  X\tilde{X} = \.{\partial} \.{X} \tilde{X} + \.{\partial} X \.{\tilde{X}} = 2^n \tilde{X} + {\gamma}_n \tilde{X}$$
where ${\gamma}_n$ are suggests to be [1, 2, 2, 0, -4, -8, -8] for $0\leq n \leq 6$ from the above and below numerical tests:

In [61]:
for n in range(1,7):
    alg = Algebra(n)
    X = create_random_multivector(alg)
    dXXr = derivative(lambda Y: X*Y.reverse(), X, alg)
    print(n, dXXr[0]/X[0])

1 2.000000000003136
2 1.999999999967596
3 -2.0622698514324334e-10
4 -3.9999999999209845
5 -8.000000001642698
6 -7.999999999946665


$\partial X^2$ is conversely neither a multiple of $X$ nor $\tilde{X}$.
But if we separate scalars or pseudoscalars from other grades, another integer multiple emerges. 
Try proving it!

In [62]:
# 2.30 var2
for n in range(2,7):
    alg = Algebra(n)
    X = create_random_multivector(alg)
    dXX = derivative(lambda X: X**2, X, alg)
    ratios = np.round(dXX[0]/X[0])
    ratiov = np.round(dXX[1]/X[1])
    ratiops = np.round(dXX[2**n-1]/X[2**n-1])
    print(ratios, ratiov, ratiops, max_diff(dXX[1:2**n-1], ratiov*(X[1:2**n-1]))) 

8.0 4.0 4.0 4.022115973612017e-12
16.0 8.0 16.0 4.148326127051405e-11
32.0 16.0 16.0 5.397993163569481e-11
64.0 32.0 64.0 2.4486723759764573e-10
128.0 64.0 64.0 4.0605385720482445e-10


In [63]:
#2.37a
from scipy.special import comb
for n, r in [(5,2), (6,2), (1,1), (1,0), (2,0)]:
    alg = Algebra(n)
    X = create_random_multivector(alg)
    print(derivative(lambda X: X, X, alg, r), comb(n, r, exact=True))

10.0 10
15.0 15
1.0 1
1.0 1
1.0 1


In [64]:
#2.38a
def gamma(n, r, s):
    gamma = 0
    K = int(np.round((r + s - abs(r - s))/2))
    for k in range(0, K+1):
        gamma += (-1)**(r*s-k)*comb(r, k, exact=True)*comb(n-r, s-k, exact=True)
    return gamma

for n, r, s in [(5,2,1), (6,2,3), (1,1,1), (1,0,1), (2,0,1)]:
    alg = Algebra(n)
    X = create_random_multivector(alg)
    Ar = create_random_multivector(alg).grade(r)
    print(max_diff(derivative(lambda X: Ar*X, X, alg, s), gamma(n, r, s)*Ar)) 

2.7938207303179752e-11
6.481482017761664e-11
1.2422285422530877e-12
2.419398015263141e-12
1.5210055437364645e-13


In [65]:
#2.38b
n = 5
s = 2
alg = Algebra(n)
X = create_random_multivector(alg)
A = create_random_multivector(alg)
max_diff(derivative(lambda X: A*X, X, alg, s), sum(gamma(n, r, s)*A.grade(r) for r in range(0, n+1)))


1.0814438233808232e-10

Construct $\gamma_n$ from $\Gamma_s^r$

In [66]:
#2.38a reverse version
def gamma_rev(n, r, s):
    gamma = 0
    K = int(np.round((r + s - abs(r - s))/2))
    for k in range(0, K+1):
        gamma += (-1)**(r*s-k+s*(s-1)/2)*comb(r, k, exact=True)*comb(n-r, s-k, exact=True)
    return gamma

for n, r, s in [(5,2,1), (6,2,3), (1,1,1), (1,0,1), (2,0,1)]:
    alg = Algebra(n)
    X = create_random_multivector(alg)
    Ar = create_random_multivector(alg).grade(r)
    dAXr = derivative(lambda X: Ar*X.reverse(), X, alg, s)
    print(max_diff(dAXr, gamma_rev(n, r, s)*Ar)) 

4.8054893397875276e-11
8.321743294459338e-11
2.1094237467877974e-14
7.771561172376096e-15
3.0477842472009797e-12


In [67]:
for n in range(7):
    r_parts = np.zeros(n+1)
    for s in range(0, n+1):
        for r in range(0, n+1):
            r_parts[r] += (-1)**(r*(r-1)/2)*gamma_rev(n, r, s) # one more reversion
    print(r_parts)

[1.]
[2. 2.]
[2. 2. 2.]
[0. 0. 0. 0.]
[-4. -4. -4. -4. -4.]
[-8. -8. -8. -8. -8. -8.]
[-8. -8. -8. -8. -8. -8. -8.]


The coefficients of each grade match due to the reversion-symmetry of gamma_rev.
Thus we get $\gamma_n$ by fixing any r:

In [68]:
def gamma_n(n):
    return sum(gamma_rev(n, 0, s) for s in range(0, n+1))
[gamma_n(n) for n in range(7)]

[1.0, 2.0, 2.0, 0.0, -4.0, -8.0, -8.0]

In [69]:
#3.1a dAG and ddGA are equivalent according to 2.18
h = 1e-3
alg = Algebra(2)
A = create_random_multivector(alg)
B = create_random_multivector(alg)
G = lambda X: (X+B) ** 2

dAG = derivative(lambda A: G(A^B), A, alg)
ddGA = derivative(lambda A_: differential(G, A^B, A_^B), A, alg)

max_diff(dAG, ddGA), dAG, ddGA

(2.1805135475005955e-10,
 3.86 + 4.21 𝐞₁ + 2.58 𝐞₂ + 1.85 𝐞₁₂,
 3.86 + 4.21 𝐞₁ + 2.58 𝐞₂ + 1.85 𝐞₁₂)

In [70]:
#3.1b a headache...
h = 1e-2 # sufficiently large to see the equivalence

ddGAB = derivative(lambda B: derivative(lambda A: G(A^B), A, alg), B, alg)
dAdBG = (
    derivative(lambda B_: derivative(lambda A_: differential(G, A^B, A_^B_), A, alg), B, alg) +
    derivative(lambda B_: differential(lambda X: derivative(lambda A_: differential(G, X, A_^B), A, alg), A^B, A^B_), B, alg)
    )
max_diff(ddGAB, dAdBG), ddGAB, dAdBG

(1.3852087477062014e-08,
 12.0 + 8.3 𝐞₁ + 3.99 𝐞₂ + 5.5 𝐞₁₂,
 12.0 + 8.3 𝐞₁ + 3.99 𝐞₂ + 5.5 𝐞₁₂)

In [71]:
#3.2b
h = 1e-5
for r in range(1, 7):
    alg = Algebra(r)
    G = lambda X: (X.grade(r)+B) ** 2
    frame = r_vector_frame(alg.frame, r)
    r_frame = r_vector_frame(reciprocal(alg.frame), r, reverse=True)
    for s in range(r):
        A = create_random_multivector(alg).grade(s)
        B = create_random_multivector(alg).grade(r-s)

        dAG = derivative(lambda A: G(A^B), A, alg, grade=s)
        BdG = 0
        for v, vr in zip(frame, r_frame):
            BdG += inner(B, vr) * (differential(G, A^B, v))
        assert_diff(dAG, BdG, 1e-8)
        

In [72]:
#3.2c Somehow the pseudoscalars are more ill conditioned, let me know if you figure out
h = 1e-2
for r in range(2,5):
    alg = Algebra(r)
    G = lambda X: (X.grade(r)+B) ** 2
    frame = r_vector_frame(alg.frame, r)
    r_frame = r_vector_frame(reciprocal(alg.frame), r, reverse=True)
    for s in range(r):
        A = create_random_multivector(alg).grade(s)
        B = create_random_multivector(alg).grade(r-s)

        dAG = lambda B: derivative(lambda A: G(A^B), A, alg, grade=s)
        dBdAG = derivative(dAG, B, alg, grade=r-s)
        BdG = 0
        for v, vr in zip(frame, r_frame):
            for u, ur in zip(frame, r_frame):
                BdG += inner(A, u) * inner(B, vr) * (differential(lambda X: differential(G, X, ur), A^B, v))
        BdG *= (-1)**(s*(r-s))
        BdG += comb(r, r-s)*derivative(G, A^B, alg, grade=r)

        print(dBdAG, BdG, sep="\n")

1.6 𝐞₁₂
1.59 𝐞₁₂
1.67 + -0.9 𝐞₁₂
1.67 + -0.9 𝐞₁₂
4.34 𝐞₁₂₃
2.94 𝐞₁₂₃
-8.88e-14 + -0.941 𝐞₁ + -0.528 𝐞₂ + 1.16 𝐞₃ + 3.18 𝐞₁₂ + 1.36 𝐞₁₃ + 4.68 𝐞₂₃ + 4.37 𝐞₁₂₃
-1.67e-13 + -0.941 𝐞₁ + -0.528 𝐞₂ + 1.16 𝐞₃ + 3.18 𝐞₁₂ + 1.36 𝐞₁₃ + 4.68 𝐞₂₃ + 4.37 𝐞₁₂₃
5.57 𝐞₁ + 1.36 𝐞₂ + 2.95 𝐞₃ + 4.94 𝐞₁₂₃
5.57 𝐞₁ + 1.36 𝐞₂ + 2.95 𝐞₃ + 4.94 𝐞₁₂₃
3.79 𝐞₁₂₃₄
2.68 𝐞₁₂₃₄
0.678 𝐞₁₂ + 0.292 𝐞₁₃ + -0.263 𝐞₁₄ + 0.672 𝐞₂₃ + -0.235 𝐞₂₄ + 0.16 𝐞₃₄ + -4.35 𝐞₁₂₃₄
0.678 𝐞₁₂ + 0.292 𝐞₁₃ + -0.263 𝐞₁₄ + 0.672 𝐞₂₃ + -0.235 𝐞₂₄ + 0.16 𝐞₃₄ + -4.35 𝐞₁₂₃₄
-3.13 + 8.25 𝐞₁₂ + 3.06 𝐞₁₃ + 3.2 𝐞₁₄ + 3.47 𝐞₂₃ + 2.13 𝐞₂₄ + 11.0 𝐞₃₄ + 19.6 𝐞₁₂₃₄
-3.13 + 8.25 𝐞₁₂ + 3.06 𝐞₁₃ + 3.2 𝐞₁₄ + 3.47 𝐞₂₃ + 2.13 𝐞₂₄ + 11.0 𝐞₃₄ + 19.6 𝐞₁₂₃₄
1.82 𝐞₁₂ + -0.858 𝐞₁₃ + -1.03 𝐞₁₄ + 2.24 𝐞₂₃ + -0.65 𝐞₂₄ + 1.58 𝐞₃₄ + -6.88 𝐞₁₂₃₄
1.82 𝐞₁₂ + -0.858 𝐞₁₃ + -1.03 𝐞₁₄ + 2.24 𝐞₂₃ + -0.65 𝐞₂₄ + 1.58 𝐞₃₄ + -6.88 𝐞₁₂₃₄


In [73]:
#3.4
h = 1e-3
alg = Algebra(r)
C = create_random_multivector(alg)
D = create_random_multivector(alg)
L = lambda X: C * X * D
dAL = lambda B: derivative(lambda A: L(A^B), A, alg, grade=s)
dBdAL = derivative(dAL, B, alg, grade=r-s)
max_diff(dBdAL, comb(r, r-s) * derivative(L, A^B, alg, grade=r))

6.225491233635694e-10

In [74]:
def multi_differential(F, X, vectors):
    from functools import reduce
    
    def apply_differential(current_func, vector):
        return lambda X: differential(current_func, X, vector)
    
    # Reduce applies the function cumulatively to the items of iterable, from left to right,
    # so as to reduce the iterable to a single value.
    return reduce(apply_differential, vectors, F)(X)

In [75]:
def multi_derivative(F, vectors):
    def apply_derivative(current_func, V):
        assert_blade(V)
        return lambda X: derivative(lambda Y: current_func(Y ^ X), V, alg, grade=V.grades[0])
    
    return reduce(apply_derivative, vectors, F)(1)

# Excercise: use frame and differential to split the derivatives. Iterate over multiindexes sounds a good idea

In [76]:
#3.5
from math import factorial
h = 1e-2
alg = Algebra(1,3)
C = create_random_multivector(alg)
D = create_random_multivector(alg)
L = lambda X: C * X * D
r = 3
n = len(alg.frame)
frame = alg.frame
r_frame = reciprocal(alg.frame)
vectors = create_r_vectors(r, alg)
X = wedge(vectors)

drL = multi_derivative(L, vectors) 
drL, factorial(r)*derivative(L, X, alg, grade=r)


(5.8 + 45.7 𝐞₁ + 44.2 𝐞₂ + 3.9 𝐞₃ + 10.1 𝐞₄ + -0.524 𝐞₁₂ + -4.36 𝐞₁₃ + -0.826 𝐞₁₄ + 0.26 𝐞₂₃ + 3.67 𝐞₂₄ + -6.72 𝐞₃₄ + 4.45 𝐞₁₂₃ + -5.12 𝐞₁₂₄ + -3.77 𝐞₁₃₄ + -0.971 𝐞₂₃₄ + -14.6 𝐞₁₂₃₄,
 5.8 + 45.7 𝐞₁ + 44.2 𝐞₂ + 3.9 𝐞₃ + 10.1 𝐞₄ + -0.524 𝐞₁₂ + -4.36 𝐞₁₃ + -0.826 𝐞₁₄ + 0.26 𝐞₂₃ + 3.67 𝐞₂₄ + -6.72 𝐞₃₄ + 4.45 𝐞₁₂₃ + -5.12 𝐞₁₂₄ + -3.77 𝐞₁₃₄ + -0.971 𝐞₂₃₄ + -14.6 𝐞₁₂₃₄)

In [77]:
# We can also expand the vector derivatives. And a linear function equals to its differential
drL = 0
r = 3
for v, vr in zip(frame, r_frame):
    for u, ur in zip(frame, r_frame):
        for w, wr in zip(frame, r_frame):
            drL += vr * u * w * L(wr^ur^v)
drL

5.8 + 45.7 𝐞₁ + 44.2 𝐞₂ + 3.9 𝐞₃ + 10.1 𝐞₄ + -0.524 𝐞₁₂ + -4.36 𝐞₁₃ + -0.826 𝐞₁₄ + 0.26 𝐞₂₃ + 3.67 𝐞₂₄ + -6.72 𝐞₃₄ + 4.45 𝐞₁₂₃ + -5.12 𝐞₁₂₄ + -3.77 𝐞₁₃₄ + -0.971 𝐞₂₃₄ + -14.6 𝐞₁₂₃₄

In [78]:
# A cleaner version of above
from itertools import permutations
drL = 0
for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
    drL += wedge(base_vectors[::-1]) * L(wedge(reci_vectors))
drL

5.8 + 45.7 𝐞₁ + 44.2 𝐞₂ + 3.9 𝐞₃ + 10.1 𝐞₄ + -0.524 𝐞₁₂ + -4.36 𝐞₁₃ + -0.826 𝐞₁₄ + 0.26 𝐞₂₃ + 3.67 𝐞₂₄ + -6.72 𝐞₃₄ + 4.45 𝐞₁₂₃ + -5.12 𝐞₁₂₄ + -3.77 𝐞₁₃₄ + -0.971 𝐞₂₃₄ + -14.6 𝐞₁₂₃₄

In [79]:
from itertools import product
def vectors_partial(F, vectors, directions):
    r = len(vectors)
    drF = 0
    for offset in product([1,-1], repeat=r):
        offset = np.array(offset)
        coef = np.prod(offset)
        offset = offset * h
        drF += 1/(2*h)**r * coef * F([a+v*d for a, v, d in zip(vectors, directions, offset)])
    return drF

In [80]:
#3.6 simplicial derivative
from itertools import permutations
h = 1e-2
def simplicial_derivative(F, vectors, alg):
    frame = alg.frame
    r_frame = reciprocal(alg.frame)
    drF = 0
    r = len(vectors)
    for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
        drF += wedge(base_vectors[::-1]) * vectors_partial(F, vectors, reci_vectors)
    return (1/factorial(r)) * drF

simplicial_derivative(lambda vectors: L(wedge(vectors)), vectors, alg) * factorial(r)

5.8 + 45.7 𝐞₁ + 44.2 𝐞₂ + 3.9 𝐞₃ + 10.1 𝐞₄ + -0.524 𝐞₁₂ + -4.36 𝐞₁₃ + -0.826 𝐞₁₄ + 0.26 𝐞₂₃ + 3.67 𝐞₂₄ + -6.72 𝐞₃₄ + 4.45 𝐞₁₂₃ + -5.12 𝐞₁₂₄ + -3.77 𝐞₁₃₄ + -0.971 𝐞₂₃₄ + -14.6 𝐞₁₂₃₄

In [81]:
#3.7a
h = 1e-2
for n, r in [(5,5),(5,4),(5,3),(6,5),(6,4),(6,3)]:
    alg = Algebra(n)
    vectors = create_r_vectors(r, alg)
    print(simplicial_derivative(lambda vectors: wedge(vectors), vectors, alg))

1.0
5.0 + -1.04e-13 𝐞₁₂ + 1.33e-13 𝐞₂₃ + -7.76e-14 𝐞₂₄ + 9.78e-13 𝐞₂₅
10.0 + -3.03e-13 𝐞₁₂ + 7.11e-14 𝐞₁₃ + -6.99e-13 𝐞₁₄ + 5.68e-14 𝐞₁₅ + -4.74e-15 𝐞₂₃ + -1.66e-14 𝐞₂₄ + 3.6e-13 𝐞₂₅ + -6.06e-13 𝐞₃₄ + 1.52e-13 𝐞₃₅ + 3.03e-13 𝐞₄₅
6.0 + -6.91e-12 𝐞₁₂ + -6.21e-11 𝐞₁₃ + -7.76e-12 𝐞₁₅ + -8.34e-11 𝐞₂₃ + 1.46e-12 𝐞₂₄ + -5.99e-11 𝐞₂₅ + 1.02e-11 𝐞₂₆ + -1.55e-11 𝐞₃₄ + -1.94e-11 𝐞₃₅ + 5.43e-11 𝐞₃₆ + -3.88e-11 𝐞₄₅ + -7.76e-12 𝐞₅₆
15.0 + -3.79e-14 𝐞₁₂ + -1.41e-11 𝐞₁₃ + -1.95e-11 𝐞₁₄ + 1.31e-11 𝐞₁₅ + -1.36e-11 𝐞₂₃ + -1.46e-11 𝐞₂₄ + 9.94e-12 𝐞₂₅ + -5.04e-12 𝐞₂₆ + -4.41e-13 𝐞₃₄ + -2.43e-12 𝐞₃₅ + -2.19e-12 𝐞₃₆ + 2.18e-12 𝐞₄₅ + -2.54e-12 𝐞₄₆ + 2.13e-12 𝐞₅₆
20.0 + -1.14e-13 𝐞₁₂ + -1.89e-13 𝐞₁₃ + -1.14e-13 𝐞₁₄ + -7.58e-14 𝐞₁₆ + -7.58e-14 𝐞₂₃ + -3.79e-14 𝐞₂₅ + 4.55e-13 𝐞₂₆ + 4.17e-13 𝐞₃₆ + 3.79e-14 𝐞₄₅ + 5.31e-13 𝐞₄₆ + -7.58e-14 𝐞₅₆


In [82]:
#3.7b X^2 is certainly not a linear function. I gave the r-derivatives of X^2 in 2.30 var2
h = 1e-3
for n, r in [(5,5),(5,4),(5,3),(6,5),(6,4),(6,3)]:
    alg = Algebra(n)
    vectors = create_r_vectors(r, alg)
    print(max_diff(simplicial_derivative(lambda vectors: wedge(vectors)**2, vectors, alg), (r+1) * wedge(vectors)))

5.779508250924881e-06
2.0045963812498346e-08
6.38151145260224e-08
6.010689003244085e-05
2.093928835988912e-06
1.3256339258943539e-08


In [83]:
#3.8 Just a ratio of factorials
r, s = 7, 4
factorial(r+s) / (factorial(r) * factorial(s)), comb(r+s, s)

(330.0, 330.0)

In [84]:
#3.9
n = 5
r = 4
h = 1e-5
alg = Algebra(n)
A_list = [create_random_multivector(alg) for _ in range(r)]
B_list = [create_random_multivector(alg) for _ in range(r)]
vectors = create_r_vectors(r, alg)
Fr = lambda vectors: sum(A_list[i] * vectors[i] * B_list[i] for i in range(r))

akpkFp = 0
akpkFn = 0
for i in range(r):
    akpkFp += Fr(vectors[:i] +[vectors[i]+vectors[i]*h] + vectors[i+1:])
    akpkFn += Fr(vectors[:i] +[vectors[i]-vectors[i]*h] + vectors[i+1:])

akpkFp *= 1/(2*h)
akpkFn *= 1/(2*h)
akpkF = akpkFp - akpkFn
akpkF, Fr(vectors)


(-1.45 + -0.994 𝐞₁ + -4.69 𝐞₂ + 4.9 𝐞₃ + 1.41 𝐞₄ + 0.0226 𝐞₅ + -1.21 𝐞₁₂ + 6.24 𝐞₁₃ + 0.0541 𝐞₁₄ + -0.352 𝐞₁₅ + 5.28 𝐞₂₃ + 7.66 𝐞₂₄ + -2.8 𝐞₂₅ + 5.03 𝐞₃₄ + 4.3 𝐞₃₅ + 1.89 𝐞₄₅ + 5.29 𝐞₁₂₃ + 13.3 𝐞₁₂₄ + 0.38 𝐞₁₂₅ + 6.56 𝐞₁₃₄ + 6.43 𝐞₁₃₅ + 4.02 𝐞₁₄₅ + 13.5 𝐞₂₃₄ + 6.92 𝐞₂₃₅ + 3.1 𝐞₂₄₅ + 8.03 𝐞₃₄₅ + 15.7 𝐞₁₂₃₄ + 8.09 𝐞₁₂₃₅ + 8.57 𝐞₁₂₄₅ + 8.97 𝐞₁₃₄₅ + 7.67 𝐞₂₃₄₅ + 6.42 𝐞₁₂₃₄₅,
 -1.45 + -0.994 𝐞₁ + -4.69 𝐞₂ + 4.9 𝐞₃ + 1.41 𝐞₄ + 0.0226 𝐞₅ + -1.21 𝐞₁₂ + 6.24 𝐞₁₃ + 0.0541 𝐞₁₄ + -0.352 𝐞₁₅ + 5.28 𝐞₂₃ + 7.66 𝐞₂₄ + -2.8 𝐞₂₅ + 5.03 𝐞₃₄ + 4.3 𝐞₃₅ + 1.89 𝐞₄₅ + 5.29 𝐞₁₂₃ + 13.3 𝐞₁₂₄ + 0.38 𝐞₁₂₅ + 6.56 𝐞₁₃₄ + 6.43 𝐞₁₃₅ + 4.02 𝐞₁₄₅ + 13.5 𝐞₂₃₄ + 6.92 𝐞₂₃₅ + 3.1 𝐞₂₄₅ + 8.03 𝐞₃₄₅ + 15.7 𝐞₁₂₃₄ + 8.09 𝐞₁₂₃₅ + 8.57 𝐞₁₂₄₅ + 8.97 𝐞₁₃₄₅ + 7.67 𝐞₂₃₄₅ + 6.42 𝐞₁₂₃₄₅)

In [104]:
#3.10
h = 1e-2

def skew_symmetrizer(F, vectors, alg):
    frame = alg.frame
    r_frame = reciprocal(alg.frame)
    drF = 0
    r = len(vectors)
    Ar = wedge(vectors)
    for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
        # If F is linear, vectors cause no difference in vectors_partial. The input is just Ar.
        # And we can allow Ar being any multivector
        drF += (Ar.sp(wedge(base_vectors[::-1]))) * vectors_partial(F, vectors, reci_vectors)
    return (1/factorial(r)) * drF

skew_symmetrizer(Fr, vectors, alg)

-1.69e-11 + -1.34e-10 𝐞₁ + -4.73e-11 𝐞₂ + 4.84e-11 𝐞₃ + -2.51e-11 𝐞₄ + -2.98e-10 𝐞₅ + 8.06e-11 𝐞₁₂ + -2.02e-10 𝐞₁₃ + -5.98e-11 𝐞₁₄ + 4.17e-11 𝐞₁₅ + 1.55e-10 𝐞₂₃ + 1.57e-10 𝐞₂₄ + -1.56e-11 𝐞₂₅ + -1.56e-10 𝐞₃₄ + -8.1e-11 𝐞₃₅ + 7.53e-11 𝐞₄₅ + 7.45e-11 𝐞₁₂₃ + -2.5e-10 𝐞₁₂₄ + -9.61e-11 𝐞₁₂₅ + -4.04e-11 𝐞₁₃₄ + -3.69e-10 𝐞₁₃₅ + 8.39e-11 𝐞₁₄₅ + -6.58e-11 𝐞₂₃₄ + -1.27e-11 𝐞₂₃₅ + -2.58e-11 𝐞₂₄₅ + 2.72e-11 𝐞₃₄₅ + -5.38e-12 𝐞₁₂₃₄ + 4.83e-11 𝐞₁₂₃₅ + -8.88e-11 𝐞₁₂₄₅ + 2.57e-11 𝐞₁₃₄₅ + -1.79e-10 𝐞₂₃₄₅ + 1.45e-10 𝐞₁₂₃₄₅

In [105]:
#3.11
h = 1e-2
C = create_random_multivector(alg)
D = create_random_multivector(alg)
L = lambda vectors: C * wedge(vectors) * D
skew_symmetrizer(L, vectors, alg), L(vectors)

(0.507 + -0.541 𝐞₁ + -2.74 𝐞₂ + -1.26 𝐞₃ + -1.41 𝐞₄ + -2.37 𝐞₅ + -5.5 𝐞₁₂ + 1.19 𝐞₁₃ + 1.87 𝐞₁₄ + -2.83 𝐞₁₅ + -2.01 𝐞₂₃ + 1.46 𝐞₂₄ + -4.6 𝐞₂₅ + 1.73 𝐞₃₄ + -2.47 𝐞₃₅ + -1.15 𝐞₄₅ + -2.85 𝐞₁₂₃ + -0.527 𝐞₁₂₄ + -6.21 𝐞₁₂₅ + 0.302 𝐞₁₃₄ + -0.966 𝐞₁₃₅ + 1.89 𝐞₁₄₅ + -0.629 𝐞₂₃₄ + -2.19 𝐞₂₃₅ + 1.99 𝐞₂₄₅ + 1.46 𝐞₃₄₅ + 0.757 𝐞₁₂₃₄ + -3.76 𝐞₁₂₃₅ + -0.914 𝐞₁₂₄₅ + -1.31 𝐞₁₃₄₅ + 0.764 𝐞₂₃₄₅ + 0.509 𝐞₁₂₃₄₅,
 0.507 + -0.541 𝐞₁ + -2.74 𝐞₂ + -1.26 𝐞₃ + -1.41 𝐞₄ + -2.37 𝐞₅ + -5.5 𝐞₁₂ + 1.19 𝐞₁₃ + 1.87 𝐞₁₄ + -2.83 𝐞₁₅ + -2.01 𝐞₂₃ + 1.46 𝐞₂₄ + -4.6 𝐞₂₅ + 1.73 𝐞₃₄ + -2.47 𝐞₃₅ + -1.15 𝐞₄₅ + -2.85 𝐞₁₂₃ + -0.527 𝐞₁₂₄ + -6.21 𝐞₁₂₅ + 0.302 𝐞₁₃₄ + -0.966 𝐞₁₃₅ + 1.89 𝐞₁₄₅ + -0.629 𝐞₂₃₄ + -2.19 𝐞₂₃₅ + 1.99 𝐞₂₄₅ + 1.46 𝐞₃₄₅ + 0.757 𝐞₁₂₃₄ + -3.76 𝐞₁₂₃₅ + -0.914 𝐞₁₂₄₅ + -1.31 𝐞₁₃₄₅ + 0.764 𝐞₂₃₄₅ + 0.509 𝐞₁₂₃₄₅)

In [87]:
def blade_split(Ar, alg):
    r = Ar.grades[0]
    projects = [P(e, Ar) for e in alg.frame[:r]]
    wed = wedge(projects)
    ratio = Ar[:][0]/wed[:][0]
    return projects, ratio

In [88]:
#3.12
r = 3
alg = Algebra(5)
locals().update(alg.blades)
Ar = create_r_blade(r, alg)
alpha = lambda vectors: wedge(vectors) | Ar
vectors, ratio = blade_split(Ar, alg)
vectors[0] *= ratio
max_diff(simplicial_derivative(alpha, vectors, alg), Ar)

2.6469298974873823e-11